In [ ]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
import pandas as pd
from lxml import etree
import time
import urllib
import urllib3
import pandas as pd
import os
import sys
import string
import csv
from selenium.common.exceptions import *
from urllib3.exceptions import *
requests.packages.urllib3.disable_warnings()

df = pd.read_csv('crawl_target.csv',encoding="utf-8")  
year = 108
str_year = str(year)
num_of_pdf = []
num_of_zip = []
num_of_doc = []
num_of_docx = []
num_of_nosuch = []
file_txt_all_list = []


def webpath(co_id, str_year):
    path = "https://doc.twse.com.tw/server-java/t57sb01?step=1&colorchg=1&co_id="+ co_id +"&year="+ str_year + "&mtype=F&"
    return path

def get_F04(path):
    browser.get(path)
#     headers = {
#         'Connection': 'close',
#         'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/75.0.3770.100 Safari/537.36'
#     }  
    headers={'Connection':'close'}
    session = requests.Session()
    session.keep_alive = False
    requests.adapters.DEFAULT_RETRIES = 5
    res = requests.get(path, verify = False, headers = headers, timeout = 30)
    selector = etree.HTML(res.text)
    F04_repoert = selector.xpath("//*[contains(text(), 'F04')]")
    F04_repoert_name = F04_repoert[0].xpath('string(.)')
    #print("年報表格檔名:", F04_repoert[0].xpath('string(.)'))
    return F04_repoert_name
    

def change_page():
    WindowHandle = browser.current_window_handle
    Title = browser.title
    allHandles = browser.window_handles
    routeWindowHandle = 0
    routeTitle = 'None'
    for handle in allHandles:
        if handle != browser.current_window_handle:
            browser.switch_to.window(handle)
            routeWindowHandle = browser.current_window_handle
            routeTitle = browser.title
            break
    
def for_file_pdf(unclass_file_name, toneyeebanhao):
    click_pdf_btn = browser.find_element_by_xpath("//*[contains(text(), 'F04')]")
    ActionChains(browser).click(click_pdf_btn).perform()   
    change_page()
    pdf_link = browser.find_element_by_xpath("//*[contains(text(), 'F04')]")
    if pdf_link == None:
        print("pdf_link is none")
        global overdownload
        overdownload == True
        time.sleep(20)   
    else:    
        for link in browser.find_elements_by_tag_name("a"):
            filename = str(toneyeebanhao).zfill(8) + ".pdf"
            filepath = "./save_crawl/"
            urllib.request.urlretrieve(link.get_attribute("href"), filepath + filename)  
            time.sleep(15)  

    
def for_other_file(co_id, unclass_file_name2, toneyeebanhao, file_classid):
    download_url = "https://doc.twse.com.tw/server-java/t57sb01?colorchg=1&step=9&kind=F&co_id="+ co_id +"&filename="+ unclass_file_name
    print("for_other_file:",type(file_classid))
    if file_classid  == 2 :
        secondfilename =".zip"
    elif file_classid  == 3:
        secondfilename =".doc"
    else:
        secondfilename =".docx"
    #print("for_other_file:",secondfilename)
    filename = str(toneyeebanhao).zfill(8) + secondfilename
    filepath = "./save_crawl/"
    urllib.request.urlretrieve(download_url, filepath + filename)
    
    
def file_class(unclass_file_name):
    if unclass_file_name.find(".pdf") != -1 :
        file_classid = 1
        num_of_pdf.append(1)
    elif unclass_file_name.find(".zip") != -1 :
        num_of_zip.append(2)
        file_classid = 2      
    elif unclass_file_name.find(".doc") != -1 :
        num_of_doc.append(3)
        file_classid = 3        
    else:
        num_of_docx.append(4)
        file_classid = 4
        
    num = file_classid
    unclass_file_name2 = unclass_file_name

    if  file_classid == 2 or file_classid == 3 or file_classid == 4:
        for_other_file(co_id, unclass_file_name2, toneyeebanhao, file_classid)
    else:
        for_file_pdf(unclass_file_name, toneyeebanhao)
        
    return file_classid, len(num_of_pdf), len(num_of_zip), len(num_of_doc), len(num_of_docx)


In [ ]:

i = 0
while i <= len(df): 
    global overdownload
    overdownload = False
    co_id = df['公司代號'][i]
    if len(co_id) == 3:
        co_id = co_id.zfill(3)
    toneyeebanhao = df['營利事業統一編號'][i]
    # chromedriver options
    options = webdriver.ChromeOptions()
    dirpath = "./save_crawl/"
    prefs = {
                'profile.default_content_settings.popups': 0,
                'download.default_directory': dirpath
    } 
    options.add_experimental_option('prefs', prefs)
    browser = webdriver.Chrome(options=options)
    browser.implicitly_wait(10)
    browser.set_window_size(700,700)
    current_link = webpath(co_id, str_year)
    file_txt_list = []  
    try:
        unclass_file_name = get_F04(current_link)
        if unclass_file_name =="":
            save_status = ""
            save_status = "沒有年報(找不到F04)"
            python_status = "沒有年報"
        else:
            save_status = ""
            save_status = "successful"
            python_status = "Fine"
            file_class(unclass_file_name)
            #print(file_class(unclass_file_name))

    except MaxRetryError as e:
            save_status = ""
            save_status = "successful"
            python_status = e
            #print("MaxRetryError",save_status)

    except BaseException as e:
            num_of_nosuch.append(co_id)
            save_status = ""
            save_status = "FAIL"
            #print("save_status NoSuchElementException ",save_status)
            python_status = e
            browser.quit()

    file_txt_list.append(time.asctime(time.localtime(time.time())))        
    file_txt_list.append(co_id)
    file_txt_list.append(toneyeebanhao)
    file_txt_list.append(save_status)
    file_txt_list.append(python_status)
    file_txt_list.append(current_link)
    file_txt_all_list.append(file_txt_list)
    print(time.asctime(time.localtime(time.time())),":",i,":",save_status) 

    with open('./log/crawl_report.csv', 'w', newline='') as f:
        writer = csv.writer(f)
        writer.writerow(('時間','company_id', '統一編號','file status','python status', 'link'))
        for it in file_txt_all_list:
            writer.writerow((column for column in it))

    if overdownload == False:
        i+=1
        time.sleep(20)
    else:
        time.sleep(600)
    browser.quit() 
print("結束:",time.asctime(time.localtime(time.time())))
